## WEBSCRAPING DE GENECARDS

GRUPO 7:
 - Salvador Cama, Alejandro 20190310
 - Velasquez Gonzales, Pedro Daniel 20170382
 - Farfán Florián Erik David 20160229
 - Inoñan Sandoval, José Agustín  20190299

In [ ]:
# Importamos las librerías necesarias para realizar el webscraping
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import re
import numpy as np

In [ ]:
# Extraemos los datos del archivo Excel
datos=pd.read_excel("CHGenesOrdenadosConVecinos.xlsx",engine="openpyxl",skiprows=1,usecols=[0,1,2,3,4,5,6,7])
# Imprimimos los datos extraídos
datos


In [ ]:
# Creando una lista con los nombres abreviados de los genes 
lista_genes = datos['Gen Abrev'].values
print(lista_genes)
# print(len(lista_genes))   # son 212 genes

In [ ]:
# Generamos los link`s correspondientes a cada gen y los almacenamos en una lista.
links=[]
# recorremos la lista de genes para crear los urls necesarios
for i in lista_genes:
    url="https://www.genecards.org/cgi-bin/carddisp.pl?gene="+i+"#expression"
    links.append(url)  #llenamos la lista con cada url
print(links)

In [ ]:
#Usamos webdriver para controlar chrome
driver=webdriver.Chrome(executable_path="C:/Users/RYZEN 5-2600/Documents/chromedriver.exe")
#CREAMOS LISTA VACIA html
html=[]
#Recorremos la lista de links de los genes
for link in links_genes:
    driver.get(link) #abre cada link
    try:            #evita errores
        element = driver.find_element_by_id("expressionImage") #recurre al motor de busqueda
        soup = BeautifulSoup(driver.page_source,features="html.parser") #crea la sopa
        while(soup.find("area")==None): #mientras area sea None, llenamos la lista html
            driver.get(link)
            soup = BeautifulSoup(driver.page_source,features="html.parser")
        html.append(soup)
    except:
        html.append(None)
        continue
driver.quit() #cerremos el motor de busqueda

In [ ]:
#Se crea una lista vacia llamada info
info = [] 
for i in html:
    L=[]  # lista vacía para almacenar los datos solicitados de los html`s
    if i == None:   # si no encuentra el html lo almacena como None
        L.append(None)
    else:  
        soup=i  # de lo contrario, si encuentra un html con informacion se comienza a agragar los datos solicitados utilizando BeautifulSoup
        L.append(soup.find('span',{'class':'gc-category'}).text)
        L.append(soup.find_all('span',{'class':'aliasMainName'})[1].text)
        lista=soup.find_all('area',{'title':re.compile('.Breast.')})
        L.append(lista[0]['title'].split(',')[0]+', '+lista[3]['title'].split(',')[0])
        L.append(lista[1]['title'].split(',')[0])
        L.append(lista[2]['title'].split(',')[0])
    info.append(L)

print(info)

In [ ]:
# Creamos el data frame "info2" con las columnas que poseen la informacion requerida
info2 = pd.DataFrame(info , columns=['Category','Name of the gene','RNASeq','Microarray','SAGE'])
# Creamos la primera columna con los nombres de los genes
columna1=datos['Gen Abrev']
# Creamos las columnas de categoria y nombre
columna2=info2.iloc[:,[0,1]]
# Creamos las columnas con los datos extraidos del excel 
columna3=datos[['Inicio','Final']]
# Creamos las columnas con la informacion del BREAST
columna4=info2.iloc[:,2:5]
# Concatenamos las columnas 
EXCEL=pd.concat([columna1,columna2,columna3,columna4],axis=1)
# Llenamos con NULL los casilleros vacios
EXCEL=EXCEL.fillna(value='Null')
# Imprimimos las primeros 20 filas
EXCEL.head(20)

**BIBLIOGRAFIA:**
 - https://github.com/prbehere93/Genecards_Webscraping/blob/main/Genecards_scraper.ipynb